In [94]:
import random
from math import floor
from scipy.sparse import csc_matrix, eye, find
import numpy as np
import time
import sys

In [95]:
class RANDU:
    def __init__(self, mul, mod,seed):
        self.mul = mul
        self.mod = mod
        self.currvalue = seed
        self.seed = seed
    def rand(self):
        self.currvalue = (self.currvalue*self.mul) % self.mod
        return self.currvalue
    def rewind(self):
        self.currvalue = self.seed

def matrixpower(M, p):
    k = M.shape[0]
    if p ==0:
        return eye(k)
    M1 = M.copy()
    for _ in range(p-1):
        M1 = M1.dot(M)
       # if p % 1000 == 0:
            #M1 = matrixmod2(M1, k)
    return M1.copy()

def matrixmod2(M, k):
    for i in range(k):
        for j in range(k):
            M[i,j] = M[i,j] % 2
    return M.copy()
class Generator:
    def __init__(self, j, k, seed):
        self.j = j
        self.k = k
        self.g = RANDU(65539, 2**31, seed)
        self.currvalue = [self.g.rand() for _ in range(k)]
        self.initial = self.currvalue.copy()
        self.seed = seed
        self.currpos = 0
    def rand(self):
        self.currvalue[self.currpos] = self.currvalue[self.currpos] ^ self.currvalue[(self.currpos-self.j) % self.k]
        self.currpos = (self.currpos+1)%self.k
        val = self.currvalue[self.currpos]
        return val
    def rewind(self):
        self.currvalue = self.initial.copy()
        self.currpos = 0
    
    def jumpahead(self, jlen):
        A = csc_matrix(([1]*(self.k+1), ([i for i in range(self.k)]+[self.k-1], [i for i in range(1,self.k)]+[0, self.k-self.j])), shape=(self.k,self.k), dtype="int")
        m, r = divmod(jlen, self.k)
        dif =  self.k - self.j - r
        if dif>0:
            B = matrixpower(A, r)
            C = B.copy()
            C = C.dot(matrixpower(A, dif))
        else:
            C = matrixpower(A, self.k - self.j)
            B = C.copy()
            B = B.dot(matrixpower(A, -dif))
        C = C + eye(self.k)
        C = matrixpower(C, m)
        D = B.dot(C)
#        D = matrixmod2(D, self.k)
        invres = 0
        res = 0
        bits = csc_matrix((self.k,1), dtype = "int")
        for _ in range(sys.getsizeof(int)):
            for k in range(self.k):
                bits[k] = self.currvalue[k]%2
                self.currvalue[k]>>=1
            b = int(D.dot(bits)[0,0]) % 2
            invres = b | (invres << 1)
        self.currvalue = self.initial.copy()
        for _ in range(sys.getsizeof(int)):
            res = (res << 1) | (invres % 2)
            invres>>=1
        return res
    def just_power(self, n):
        A = csc_matrix(([1]*(self.k+1), ([i for i in range(self.k)]+[self.k-1], [i for i in range(1,self.k)]+[0, self.k-self.j])), shape=(self.k,self.k), dtype="int")
        matrixpower(A,n)

In [99]:
seed = 24121
jlen = 200000
gen = Generator(24, 55, seed)

In [8]:
gen.rewind()

In [100]:
start = time.time()
a = gen.jumpahead(jlen)
print(a)
print("it takes {} seconds".format(time.time() - start))

OverflowError: cannot convert float infinity to integer

In [98]:
start = time.time()
for _ in range(jlen-1):
    gen.rand()
print("it takes {} seconds".format(time.time() - start))
gen.rand()

it takes 0.002056598663330078 seconds


1219056257

In [75]:
start = time.time()
gen.just_power(jlen)
print("it takes {} seconds".format(time.time() - start))

it takes 0.41176486015319824 seconds


### Основная идея быстрого возведения в степень:
### Матрица $A$ является фробениусовой матрицей, внизу единица стоит в столбце с номером k-j, если считать от нуля.
### Её характеристический многочлен $p(x) = x^k-x^{k-j}-1$.
### По теореме Гамильтона-Кэли $p(A) = A^k-A^{k-j}-E=0$
### Тогда $A^k = A^{k-j}+E$
### Тогда возведение в степень $n, \ r = n \% k, \ m = n // k$
### $A^{n} = A^{k*m+r} = A^r(A^k)^m = A^r(A^{k-j}+E)^m$